# 2-1: Dense Layers

## Code.2-1-1: Shapes of Dense Layers

In [18]:
import tensorflow as tf

from tensorflow.keras.layers import Dense

N, n_feature = 1, 10  # N은 미니배치에서 데이터샘플 수 (여기서는 1개) / row(X(vector)) 길이가 10개
X = tf.random.normal(shape=(N, n_feature))

n_neuron = 3  # Layer안에 뉴런 3개
dense = Dense(units=n_neuron, activation='sigmoid')
Y = dense(X)  # 각 뉴런의 weights는 10개 bias는 1개 / W(matrix)는 row:10, column:3 이고 B(vector)는 3개를 가진다.

W, B = dense.get_weights()  # weights, bias 추출

print('===== Input/Weight/Bias =====')
print('X: ', X.shape) # 1 x 10 vector (row vector)
print('W: ', W.shape) # 10 x 3 matrix
print('B: ', B.shape) # 1차원 vector
print('Y: ', Y.shape) # row vector 반환

===== Input/Weight/Bias =====
X:  (1, 10)
W:  (10, 3)
B:  (3,)
Y:  (1, 3)


In [19]:
# 위 코드에서 N의 개수를 바꿨을 때 
import tensorflow as tf

from tensorflow.keras.layers import Dense

N, n_feature = 8, 10  # N은 미니배치에서 데이터샘플 수 (여기서는 8개) / row(X(vector)) 길이가 10개
X = tf.random.normal(shape=(N, n_feature))

n_neuron = 3  # Layer안에 뉴런 3개
dense = Dense(units=n_neuron, activation='sigmoid')
Y = dense(X)  # 각 뉴런의 weights는 10개 bias는 1개 / W(matrix)는 row:10, column:3 이고 B(vector)는 3개를 가진다.

W, B = dense.get_weights()  # weights, bias 추출

print('===== Input/Weight/Bias =====')
print('X: ', X.shape) # 8 x 10 matrix
print('W: ', W.shape) # 10 x 3 matrix
print('B: ', B.shape) # 1차원 vector
print('Y: ', Y.shape) # 8 x 3 matrix 반환

===== Input/Weight/Bias =====
X:  (8, 10)
W:  (10, 3)
B:  (3,)
Y:  (8, 3)


## Code.2-1-2: Output Calculations

In [20]:
import numpy as np
import tensorflow as tf

from tensorflow.math import exp
from tensorflow.linalg import matmul

from tensorflow.keras.layers import Dense

N, n_feature = 4, 10  # N은 미니배치에서 데이터샘플 수 (여기서는 4개) / row(X(vector)) 길이가 10개
X = tf.random.normal(shape=(N, n_feature))

n_neuron = 3  # Layer안에 뉴런 3개
dense = Dense(units=n_neuron, activation='sigmoid')
Y_tf = dense(X)  # 각 뉴런의 weights는 10개 bias는 1개 / W(matrix)는 row:10, column:3 이고 B(vector)는 3개를 가진다.

W, B = dense.get_weights()  # weights, bias 추출
print("Y(Tensorflow): \n", Y_tf.numpy())

# calculate with matrix multiplication
Z = matmul(X, W) + B  # Affine transformation
Y_man_matmul = 1/(1 + exp(-Z))  
print("Y(with matrix multiplication: \n", Y_man_matmul.numpy())

# calculate with dot products
Y_man_vac = np.zeros(shape=(N, n_neuron))
# print(Y_man_vac)
for x_idx in range(N):
  x = X[x_idx]  # X를 row로 뽑고

  for nu_idx in range(n_neuron):
    w, b = W[:, nu_idx], B[nu_idx]  # W를 column으로 뽑고, B하나 뽑고

    z = tf.reduce_sum(x * w) + b  # affine transformation
    a = 1/(1 + np.exp(-z))  # sigmoid 연산
    Y_man_vac[x_idx, nu_idx] = a  

print("Y(with dot products): \n", Y_man_vac)

Y(Tensorflow): 
 [[0.67300737 0.43524355 0.12094662]
 [0.18396258 0.7238414  0.42468086]
 [0.5834298  0.05641314 0.34350348]
 [0.17845228 0.83742905 0.15266967]]
Y(with matrix multiplication: 
 [[0.67300737 0.4352435  0.12094666]
 [0.18396258 0.72384137 0.42468086]
 [0.58342975 0.05641318 0.34350345]
 [0.17845227 0.83742905 0.15266967]]
Y(with dot products): 
 [[0.67300737 0.43524352 0.12094666]
 [0.18396259 0.72384142 0.42468086]
 [0.58342983 0.05641318 0.34350342]
 [0.17845229 0.83742906 0.15266965]]


# 2-2: Cascaded Dense Layers

## Code.2-2-1: Shape of Cacaded Dense Layers

In [24]:
import tensorflow as tf

from tensorflow.keras.layers import Dense

N, n_feature = 4, 10  # 4:minibatch size 
X = tf.random.normal(shape=(N, n_feature))

n_neurons = [3, 5] 
dense1 = Dense(units=n_neurons[0], activation='sigmoid')  # 첫번째 레이어에서는 3개의 뉴런
dense2 = Dense(units=n_neurons[1], activation='sigmoid')  # 두번째 레이어에서는 5개의 뉴런

# forward propagation
A1 = dense1(X)
Y = dense2(A1)

# get weight/bias
W1, B1 = dense1.get_weights()
W2, B2 = dense2.get_weights()

# input
print("X: {}\n".format(X.shape))  # 4 x 10 matrix

# 첫번째 Layer 관련 weights / bias / A1
print("W1: ", W1.shape) # 10 x 3 matrix 
print("B1: ", B1.shape) # 뉴런의 개수
print("A1: {}\n".format(A1.shape))  # 4 x 3 matrix  첫번째 레이어에 3개의 뉴런이 있어서 3이 나옴

# 두번째 Layer 관련 weights / bias / Y
print("W2: ", W2.shape) # 3 x 5 matrix  첫번째 Layer의 결과가 두번째 Layer에 들어가서 W: 3 x 5 matrix (3개마다 weight가 곱해져야해서 3, 그리고 뉴런이 5개여서 5)
print("B2: ", B2.shape) # 뉴런의 개수
print("Y: {}".format(Y.shape))  # 4 x 5 matrix  두번째 레이어에 5개의 뉴런이 있어서 5가 나옴

X: (4, 10)

W1:  (10, 3)
B1:  (3,)
A1: (4, 3)

W2:  (3, 5)
B2:  (5,)
Y: (4, 5)


## Code.2-2-2: Dense Layers with Python List

In [31]:
import tensorflow as tf

from tensorflow.keras.layers import Dense

N, n_feature = 4, 10  # 4:minibatch size 
X = tf.random.normal(shape=(N, n_feature))

n_neurons = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

# for문으로 Dense Layer 여러개 만들기
dense_layers = list()
for n_neuron in n_neurons:
  #print(n_neuron)
  dense = Dense(units=n_neuron, activation='relu')  # 10개의 Dense Layers가 만들어졌다.
  dense_layers.append(dense)

"""
# for문으로 Dense Layer에 값 넣기
for dense in dense_layers:  # dense_layers [dense1, dense2, ..., dense10] 10개가 있는데 dense가 반복문 한번씩마다 하나씩을 가지고 for문을 돔
  X = dense(x)  # 리스트 첫 dense1에 X를 통과시킨 값을 X에 넣고 반복될 때마다 X가 다음 dense2 ... 들어감
"""
# 확인을 위에 위 코드는 주석처리
print("Input: ", X.shape)
for dense_idx, dense in enumerate(dense_layers):
  X = dense(X)
  print("After dense layer ", dense_idx+1)
  print(X.shape, '\n')
Y = X

Input:  (4, 10)
After dense layer  1
(4, 10) 

After dense layer  2
(4, 20) 

After dense layer  3
(4, 30) 

After dense layer  4
(4, 40) 

After dense layer  5
(4, 50) 

After dense layer  6
(4, 60) 

After dense layer  7
(4, 70) 

After dense layer  8
(4, 80) 

After dense layer  9
(4, 90) 

After dense layer  10
(4, 100) 



## Code.2-2-3: Output Calculations

In [40]:
import tensorflow as tf

from tensorflow.math import exp
from tensorflow.linalg import matmul

from tensorflow.keras.layers import Dense

N, n_feature = 4, 10  # 4:minibatch size 
X = tf.random.normal(shape=(N, n_feature))
X_cp = tf.identity(X) # 다른 이름으로 똑같은걸 저장할때 사용

n_neurons = [3, 4, 5]

# for문으로 Dense Layer 여러개 만들기
dense_layers = list()
for n_neuron in n_neurons:
  #print(n_neuron)
  dense = Dense(units=n_neuron, activation='sigmoid')  # 10개의 Dense Layers가 만들어졌다.
  dense_layers.append(dense)

# forward propagation(Tensorflow)
W, B = list(), list()
for dense_idx, dense in enumerate(dense_layers):
  X = dense(X)
  w, b =dense.get_weights()

  W.append(w)
  B.append(b)

print("Y(Tensorflow): \n", X.numpy()) # row는 4(minibatch size), column은 5(neuron의 마지막)

# forward propagation(Manual)
for layer_idx in range(len(n_neurons)):
  w, b = W[layer_idx], B[layer_idx]

  X_cp = matmul(X_cp, w) + b
  X_cp = 1/(1 + exp(-X_cp))
print("Y(Manual): \n", X_cp.numpy())

Y(Tensorflow): 
 [[0.42840672 0.32897782 0.30666095 0.40006918 0.7248291 ]
 [0.43165785 0.33445147 0.31892464 0.40881258 0.7198756 ]
 [0.4309621  0.3333282  0.31326365 0.40375674 0.72078234]
 [0.42921177 0.33041582 0.32133102 0.41459507 0.7235818 ]]
Y(Manual): 
 [[0.4284067  0.3289778  0.30666092 0.40006918 0.724829  ]
 [0.4316578  0.33445144 0.31892464 0.40881255 0.7198756 ]
 [0.4309621  0.33332813 0.31326362 0.40375674 0.72078234]
 [0.42921174 0.3304158  0.321331   0.41459507 0.72358185]]
